In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
people = pd.read_csv('people.csv')
train = pd.read_csv('act_train.csv')

In [3]:
# Remove type label from people dataset
for d in range(1,10):
    people['char_%d' % d] = people['char_%d' % d].str.strip('type ').astype(int)
    
# Rename columns
people['group_1'] = people['group_1'].str.strip('group ')
people.columns = [people.columns[0]] + ['p_' + c for c in people.columns[1:]]

# Remove type label from people dataset
train.fillna('0', inplace=True)
for d in range(1,11):
    train['char_%d' % d] = train['char_%d' % d].str.strip('type ').astype(int)
train['activity_category'] = train['activity_category'].str.strip('type ').astype(int)

In [4]:
merged = pd.merge(people, train, how='inner', on='people_id')

In [5]:
merged.activity_id.nunique()-merged.shape[0]


0

In [6]:
merged.set_index(merged.activity_id, inplace=True)
merged.head()

,people_id,p_char_1,p_group_1,p_char_2,p_date,p_char_3,p_char_4,p_char_5,p_char_6,p_char_7,...,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
activity_id,,,,,,,,,,,,,,,,,,,,,
act2_1734928,ppl_100,2,17304,2,2021-06-29,5,5,5,3,11,...,0,0,0,0,0,0,0,0,76,0
act2_2434093,ppl_100,2,17304,2,2021-06-29,5,5,5,3,11,...,0,0,0,0,0,0,0,0,1,0
act2_3404049,ppl_100,2,17304,2,2021-06-29,5,5,5,3,11,...,0,0,0,0,0,0,0,0,1,0
act2_3651215,ppl_100,2,17304,2,2021-06-29,5,5,5,3,11,...,0,0,0,0,0,0,0,0,1,0
act2_4109017,ppl_100,2,17304,2,2021-06-29,5,5,5,3,11,...,0,0,0,0,0,0,0,0,1,0


In [7]:
merged.columns

Index([u'people_id', u'p_char_1', u'p_group_1', u'p_char_2', u'p_date',
       u'p_char_3', u'p_char_4', u'p_char_5', u'p_char_6', u'p_char_7',
       u'p_char_8', u'p_char_9', u'p_char_10', u'p_char_11', u'p_char_12',
       u'p_char_13', u'p_char_14', u'p_char_15', u'p_char_16', u'p_char_17',
       u'p_char_18', u'p_char_19', u'p_char_20', u'p_char_21', u'p_char_22',
       u'p_char_23', u'p_char_24', u'p_char_25', u'p_char_26', u'p_char_27',
       u'p_char_28', u'p_char_29', u'p_char_30', u'p_char_31', u'p_char_32',
       u'p_char_33', u'p_char_34', u'p_char_35', u'p_char_36', u'p_char_37',
       u'p_char_38', u'activity_id', u'date', u'activity_category', u'char_1',
       u'char_2', u'char_3', u'char_4', u'char_5', u'char_6', u'char_7',
       u'char_8', u'char_9', u'char_10', u'outcome'],
      dtype='object')

In [49]:
col_drop = ['people_id','activity_id']
col_not_dummy = []
for c in merged.columns: 
    nuni = merged[c].nunique()
    if nuni<=2 or nuni>52:
        col_not_dummy.append(c)

In [50]:
col_not_dummy

['p_char_1',
 'p_group_1',
 'p_date',
 'p_char_10',
 'p_char_11',
 'p_char_12',
 'p_char_13',
 'p_char_14',
 'p_char_15',
 'p_char_16',
 'p_char_17',
 'p_char_18',
 'p_char_19',
 'p_char_20',
 'p_char_21',
 'p_char_22',
 'p_char_23',
 'p_char_24',
 'p_char_25',
 'p_char_26',
 'p_char_27',
 'p_char_28',
 'p_char_29',
 'p_char_30',
 'p_char_31',
 'p_char_32',
 'p_char_33',
 'p_char_34',
 'p_char_35',
 'p_char_36',
 'p_char_37',
 'p_char_38',
 'date',
 'char_10',
 'outcome']

In [51]:
merged.drop(col_drop, axis=1, inplace=True)


ValueError: labels ['people_id' 'activity_id'] not contained in axis

In [68]:
print len(set(merged.columns)-set(col_not_dummy))

18


In [69]:
features = merged[col_not_dummy]
for c in set(merged.columns)-set(col_not_dummy):
    features=pd.concat([features,pd.get_dummies(merged[c],c)], axis=1)
    print('%s done' % c)

p_char_6 done
p_char_7 done
p_char_4 done
char_4 done
p_char_2 done
p_char_3 done
p_char_8 done
p_char_9 done
p_char_5 done
char_9 done
char_8 done
char_7 done
char_6 done
char_5 done
activity_category done
char_3 done
char_2 done
char_1 done


In [71]:
features.head()

,p_char_1,p_group_1,p_date,p_char_10,p_char_11,p_char_12,p_char_13,p_char_14,p_char_15,p_char_16,...,char_1_42,char_1_43,char_1_44,char_1_45,char_1_46,char_1_47,char_1_48,char_1_49,char_1_50,char_1_52
activity_id,,,,,,,,,,,,,,,,,,,,,
act2_1734928,2,17304,2021-06-29,True,False,False,True,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
act2_2434093,2,17304,2021-06-29,True,False,False,True,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
act2_3404049,2,17304,2021-06-29,True,False,False,True,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
act2_3651215,2,17304,2021-06-29,True,False,False,True,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
act2_4109017,2,17304,2021-06-29,True,False,False,True,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
features.to_csv('./data/features_first_set.csv')

KeyboardInterrupt: 